# Python: JSON Object Serialization

Python allows objects to be serialized in a JSON format which is highly portable across applications.
This is possible through the usage of the json module, which is specialized in saving dictionary objects into JSON format. 

The json module also allows usage of custom encoders/decoders to be used for serialization of complex objects.

In [1]:
# importing the module
import json

## Serializing dictionary data in JSON format

The json module works first and foremost with dictionary data. It is able to serialize dictionary data with almost no requirement for development customization.

In [2]:
# creating dictionary data for representing domain objects
# such data can be processed out of the box by the JSON module
dict_fridge_product = {
    "name": "Fridge"
}

dict_television_set_product = {
    "name": "Television Set"
}

dict_electric_product_category = {
    "name": "Electric Products",
    "sub_categories": [],
    "products": [dict_fridge_product, dict_television_set_product]
}

dict_entertainment_product_category = {
    "name": "Entertainment Product",
    "sub_categories": [],
    "products": [dict_television_set_product]
}

dict_taxed_product_category = {
    "name": "Taxed Products",
    "sub_categories": [dict_electric_product_category, dict_entertainment_product_category]
}

In [3]:
# we can save the top level product category information into a string
# and then reload it

# save the json data and used a 2 spaces indent for readability
json_string = json.dumps(dict_taxed_product_category, indent=2)
print("The generated json string is: \n {0} \n".format(json_string))

#load back the object from the JSON string
loaded_dict_taxed_product_category = json.loads(json_string)

# display the loaded data for comparison with original data
print(
    "The loaded dict_taxed_product_category has type {0} and its content is: \n {1} \n".format(
        type(loaded_dict_taxed_product_category),
        loaded_dict_taxed_product_category
    )
)

The generated json string is: 
 {
  "name": "Taxed Products",
  "sub_categories": [
    {
      "name": "Electric Products",
      "sub_categories": [],
      "products": [
        {
          "name": "Fridge"
        },
        {
          "name": "Television Set"
        }
      ]
    },
    {
      "name": "Entertainment Product",
      "sub_categories": [],
      "products": [
        {
          "name": "Television Set"
        }
      ]
    }
  ]
} 

The loaded dict_taxed_product_category has type <class 'dict'> and its content is: 
 {'name': 'Taxed Products', 'sub_categories': [{'name': 'Electric Products', 'sub_categories': [], 'products': [{'name': 'Fridge'}, {'name': 'Television Set'}]}, {'name': 'Entertainment Product', 'sub_categories': [], 'products': [{'name': 'Television Set'}]}]} 



In [4]:
# JSON serialized data can be also saved in files
# and later on dictionaries can be recreated from this data

# save data to file
json_file_write = open("data.json", "w")
json.dump(dict_taxed_product_category, json_file_write, indent=2)
json_file_write.close()

# load data from file
json_file_read = open("data.json", "r")
file_loaded_dict_taxed_product_category = json.loads(json_string)
json_file_read.close()

# display the loaded data for comparison with original data
print(
    "The loaded dict_taxed_product_category has type {0} and its content is: \n {1} \n".format(
        type(file_loaded_dict_taxed_product_category),
        file_loaded_dict_taxed_product_category
    )
)

The loaded dict_taxed_product_category has type <class 'dict'> and its content is: 
 {'name': 'Taxed Products', 'sub_categories': [{'name': 'Electric Products', 'sub_categories': [], 'products': [{'name': 'Fridge'}, {'name': 'Television Set'}]}, {'name': 'Entertainment Product', 'sub_categories': [], 'products': [{'name': 'Television Set'}]}]} 



## Serializing objects in JSON format

The json module can serialize complex objects in JSON format as well. In this case, it will require implementation of custom encoders and decoders that will ensure the transformation of objects to and from dictionary data. 

In [5]:
# creating data to serialize, we will use simple code
class Product:

    def __init__(self):
        self.name  = None

class ProductCategory :
    
    def __init__(self):
        self.name = None
        self.categories = list([])
        self.products = []

# initial product data creation
fridge_product = Product()
fridge_product.name = "Fridge"

television_set_product = Product()
television_set_product.name = "Television Set"

# initial category data creation
electric_product_category = ProductCategory()
electric_product_category.name = "Electric Products"
electric_product_category.products.append(fridge_product)
electric_product_category.products.append(television_set_product)

entertainment_product_category = ProductCategory()
entertainment_product_category.name = "Entertainment Product"
entertainment_product_category.products.append(television_set_product)

taxed_product_category = ProductCategory()
taxed_product_category.name = "Taxed Products"
taxed_product_category.categories.append(electric_product_category)
taxed_product_category.categories.append(entertainment_product_category)

In [6]:
# ensure product encoding
class ProductEncoder(json.JSONEncoder):
    def default(self, o):
        # if the object is a product we will simply provide the dictionary
        if type(o) == Product:
            data = o.__dict__.copy()
            # we store a product data type
            data["__data_type"] = repr(Product)
            return data
        else:
            # otherwise we return None
            return None

# ensure product decoding


class ProductDecoder(json.JSONDecoder):
    def __init__(self):
        # register object hook
        super().__init__(object_hook=self.object_hook)

    # the object hook will ensure that products are decoded
    def object_hook(self, dict):
        # if we don't receive a dictionary
        # we return the same object
        if (type(dict) != type({})):
            return dict

        # if the dictionary has the current value marked as product
        # we will extract and return the product
        if (repr(Product) == dict["__data_type"]):
            product = Product()
            product.name = dict["name"]
            return product
        else:
            # otherwise we return the same dictionary
            return dict

In [7]:
# ensure product category encoding
class ProductCategoryEncoder(json.JSONEncoder):
    def default(self, o):
        # if the object is a product category
        # we will create a custom dictionary
        if type(o) == ProductCategory:
            data = {}
            data["name"] = o.name

            # add categories information
            data["categories"] = list()
            for product_category in o.categories:
                data["categories"].append(
                    ProductCategoryEncoder().default(product_category))

            # add products information
            data["products"] = list()
            for product in o.products:
                data["products"].append(ProductEncoder().default(product))

            # add product category data marker
            data["__data_type"] = repr(ProductCategory)
            return data
        else:
            # otherwise we return None
            return None


# ensure product category decoding
class ProductCategoryDecoder(json.JSONDecoder):
    def __init__(self):
        # register object hook
        super().__init__(object_hook=self.object_hook)

    # the object hook will ensure that products are decoded
    def object_hook(self, dict):
        # if we don't receive a dictionary
        # we return the same object
        if (type(dict) != type({})):
            return dict

        # if the dictionary has the current value marked as product
        # we will extract and return the product
        if (repr(ProductCategory) == dict["__data_type"]):
            product_category = ProductCategory()
            product_category.name = dict["name"]

            if "categories" in dict:
                product_category.categories = []
                for category_data in dict["categories"]:
                    product_category.categories.append(
                        ProductCategoryDecoder().object_hook(category_data))

            if "products" in dict:
                product_category.products = []
                for product_data in dict["products"]:
                    product_category.products.append(
                        ProductDecoder().object_hook(product_data))

            return product_category
        else:
            # otherwise we return the same dictionary
            return dict

In [8]:
# generate JSON string
object_json_string = json.dumps(taxed_product_category, cls = ProductCategoryEncoder, indent= 2)
print("The generated JSON string is: \n{0}\n".format(object_json_string))

The generated JSON string is: 
{
  "name": "Taxed Products",
  "categories": [
    {
      "name": "Electric Products",
      "categories": [],
      "products": [
        {
          "name": "Fridge",
          "__data_type": "<class '__main__.Product'>"
        },
        {
          "name": "Television Set",
          "__data_type": "<class '__main__.Product'>"
        }
      ],
      "__data_type": "<class '__main__.ProductCategory'>"
    },
    {
      "name": "Entertainment Product",
      "categories": [],
      "products": [
        {
          "name": "Television Set",
          "__data_type": "<class '__main__.Product'>"
        }
      ],
      "__data_type": "<class '__main__.ProductCategory'>"
    }
  ],
  "products": [],
  "__data_type": "<class '__main__.ProductCategory'>"
}



In [9]:
# defining a function for recursively
# print product category data
def print_product_category(category, level = 0):
    print(level*"\t", "Category name:", category.name)

    for product in category.products:
        print(level*"\t", " ", "Product name:", product.name)
            
    for subcategory in category.categories:
        print_product_category(subcategory, level + 1)

In [10]:
# load product category from string
loaded_product_category = json.loads(object_json_string, cls = ProductCategoryDecoder)
print_product_category(loaded_product_category)

 Category name: Taxed Products
	 Category name: Electric Products
	   Product name: Fridge
	   Product name: Television Set
	 Category name: Entertainment Product
	   Product name: Television Set


In [11]:
# save a product category object into a json file
object_file_write = open("object_data.json", "w")
json.dump(taxed_product_category, object_file_write, cls = ProductCategoryEncoder, indent= 2)
object_file_write.close()

# load the product category object from the json file
object_file_read = open("object_data.json", "r")
file_loaded_taxed_product_category = json.load(object_file_read, cls = ProductCategoryDecoder)
object_file_read.close()

# print loaded object's values
print_product_category(file_loaded_taxed_product_category)

 Category name: Taxed Products
	 Category name: Electric Products
	   Product name: Fridge
	   Product name: Television Set
	 Category name: Entertainment Product
	   Product name: Television Set
